In [1]:

#First Let's Import all the needed libraries for the project

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import tweepy
import os
import json
import time
import datetime
import warnings



%matplotlib inline

# Gathering Data


### 1. Using pandas read_csv

In [4]:
# Here will start Gathering of data using Pandas read_csv
df_twitter_csv = pd.read_csv('twitter-archive-enhanced.csv')
# df_twitter_csv.loc[(df_twitter_csv.floofer == 'floofer') & (df_twitter_csv.puppo == 'puppo')]
df_twitter_csv.shape

(2356, 17)

### 2. Using Request library to download file

In [5]:
# Now will gather the image predictions using request library to download file programmatically
image_pred_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(image_pred_url)
filename = image_pred_url.split('/')[-1]

# write the file with the response content 
if not os.path.isfile(filename):
    with open('image_predictions.tsv', 'wb') as file:
        file.write(response.content)
        
# read the file to a pandas dataframe 
df_image_pred = pd.read_csv('image_predictions.tsv',sep = '\t')
df_image_pred.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


### 3. Using Twitter API

In [ ]:
# to use the api will authenticate my developer user 
# and then use tweepy to query the data

consumer_key = 'dNFaYWsrGMWUfPa1sWUYKCDsx'
consumer_secret = '9sxwSJCxyJMaeHduNgxkW0FbEo0rd8MfuW1kJeQI2jyjpSuCOy'
access_token = '403356258-RcWDMomy4G7QqlMBnbSL0v0g4jge8x6NhOuAY9M3'
access_secret = 'Jf4LkOzGyXcU2glEQ54QyT9VS2TeWM19FaH7z6q3q99WV'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth,wait_on_rate_limit = True)
# test
tweet = api.get_status(df_twitter_csv.tweet_id[3], tweet_mode='extended')
print(tweet._json['full_text'])


In [20]:
# create a txt file to save the json data in
txtfile = 'tweet_json.txt'

if not os.path.isfile(txtfile):
    open(txtfile, 'w', encoding = 'UTF-8')
    
tweets = df_twitter_csv['tweet_id'] 
tweets.count()

2356

In [ ]:
# Loop over tweet_ids to query each tweet data and fill the text file with json data
# and create a list with any tweet cause error

errors = []
starttime = time.time()

if not os.path.isfile(txtfile):
    with open('tweet_json.txt', 'w', encoding = 'UTF-8') as file:
        for tweet_id in tweets:
         
            try:  
                status = api.get_status(tweet_id, tweet_mode = 'extended')
                json.dump(status._json, file)
                file.write('\n')
            except Exception as e:
                print('Tweet ID causes Error :', tweet_id , '; Error : ',str(e))
                errors.append(tweet_id)
#             .args[0][0]['message']
endtime = time.time()
print('Consumed time:', endtime - starttime)

In [ ]:
df_tweets = []
dict_tweets = []

with open('tweet_json.txt','r') as js_file:
    # data = json.loads(json_file)
    for line in js_file:
        jsobject = json.loads(line)
        df_tweets.append({'tweet_id': jsobject['id'],
                        'retweet_count': jsobject['retweet_count'],
                        'favorite_count': jsobject['favorite_count']})


df_twitter_api = pd.DataFrame(df_tweets, columns= ['tweet_id',
                                           'retweet_count',
                                           'favorite_count'])  
print(df_twitter_api.count())
df_twitter_api.head()


# Assessing Data

### We gathered 3 dataframes that need to be assessed :
* df_twitter_csv
* df_image_pred
* df_twitter_api

### 1. df_twitter_csv :

will assess the data visually on excel spreadsheet

In [ ]:
df_twitter_csv.info()
df_twitter_csv.shape

In [ ]:
df_twitter_csv.head()


In [ ]:
df_twitter_csv.describe()


In [ ]:
df_twitter_csv.name.value_counts()

In [ ]:
df_twitter_csv.rating_numerator.sort_values()

In [ ]:
# # checking for the number of observations where rating numerator is smaller than 6
print(df_twitter_csv.rating_numerator[df_twitter_csv['rating_numerator'] < 6  ].count())
df_twitter_csv.rating_numerator[df_twitter_csv['rating_numerator'] < 6  ]

In [ ]:
# # checking for the number of observations where rating numerator is greater than 15 
print(df_twitter_csv.rating_numerator[df_twitter_csv['rating_numerator'] > 15  ].count())
df_twitter_csv.rating_numerator[df_twitter_csv['rating_numerator'] > 15  ]

In [ ]:
# All the numerator values greater than 20 while the denominator is also 10 which indicated a faulty value of numerator
num_out_range_df = df_twitter_csv.query('(rating_numerator > 20  ) and rating_denominator == 10 ' )
print(num_out_range_df.rating_numerator.count())
num_out_range_df.head(9)

In [ ]:
# rating denominator sorted values
df_twitter_csv.rating_denominator.sort_values()

In [ ]:
# checking for the number of observations where rating denominator not equal 10  
df_twitter_csv.rating_denominator[df_twitter_csv['rating_denominator'] != 10 ].count()

In [ ]:
# checking for observations where rating denominator not equal 10  
df_twitter_csv.rating_denominator[df_twitter_csv['rating_denominator'] != 10 ]

In [ ]:
# Checking for duplicates in the csv dataframe
df_twitter_csv.duplicated().sum()

####  2. Image Prediction dataframe

In [ ]:
# random 5 samples from the dataframe 
df_image_pred.sample(5)

In [ ]:
df_image_pred.info()

In [ ]:
# export the prediction dataframe to a csv file to assess it visually on microsoft excel
df_image_pred.to_csv('df_image_pred.csv')

In [ ]:
df_image_pred.describe()

In [ ]:
df_image_pred.p1.unique()

In [ ]:
df_image_pred.p1.value_counts()

In [ ]:
df_image_pred.duplicated().sum()

### 3. Twitter API Dataframe

#### Data Will be visually assessed in microsoft excel sheet 

In [ ]:
df_twitter_api.info()

In [ ]:
df_twitter_api.sample(5)

In [ ]:
df_twitter_api.duplicated().sum()

## Assessment Summary

### Quality Issues

### 1.Completeness Issues :

##### a. df_twitter_csv :
* some tweets are just retweets on the original tweets that need to b removed.
* Some tweets has no images (number of image predictions are not compatible with archive dataframe)
* Missing values in dog names Column that failed to be extracted from the text.
* Missing values in Name column are "None" not nan and also in dog classifications columns.



##### b. df_image_pred :
* Column names are not descriptive 


##### c. df_twitter_api :
* number of rows is not consistent with the archive data number of tweets which means not all tweets have retweets or favorite count.


### 2.Inconsistency  Issues :

##### a. df_twitter_csv :
* Capitalization inconsistency in predicted breed names.
* Timestamp is of object type not date type

### 2.Accuracy  Issues :

##### a. df_twitter_csv :
* rating numerators have some abnormal values up to 1776 which is not normal and need investigation
* rating denominators have some abnormal data (while denominators should always be 10 ) for each dog observation.


### Tidiness Issues:

##### a. df_twitter_csv :
* variables are used as column names (doggo,floofer,pupper,puppo) .

##### b. df_image_pred:
* unnecessary column existing that will not help the analysis (image_num: which is just number 

# 3. Cleaning Data

In [ ]:
df_csv_clean = df_twitter_csv.copy()
df_pred_clean = df_image_pred.copy()
df_api_clean = df_twitter_api.copy()

## 1. Completeness

### a. df_twitter_csv :

* ### a.1.1 Define

- we should remove retweets and replies from df_csv_clean

* ### Code

In [ ]:
df_csv_clean.info()
df_csv_clean = df_csv_clean[(df_csv_clean['in_reply_to_status_id'].isna() == True)  & (df_csv_clean['retweeted_status_id'].isna() == True)]
df_csv_clean.drop(columns  = ['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id',
                              'retweeted_status_timestamp'],inplace = True)


* ### Test

In [ ]:
df_csv_clean.info()

* ### a.1.2 Define

- Not all the tweets have images using image prediction dataframe will align the rows of all 3 dataframes which only have photos

* ### Code

In [ ]:
master_df_clean = pd.merge(df_csv_clean, df_pred_clean,
                                   how = 'inner', on = 'tweet_id')


master_df_clean.to_csv('master_clean.csv')

* ### Test

In [ ]:
master_df_clean.info()

* ### a.1.3 Define

number of rows of api dataframe is not consistent with the archive data number of tweets which means not all tweets have retweets or favorite count.

* ### Code

In [ ]:
# merge 'master_df_clean' and 'tweets_df_clean' and also inner join in order to exclude rows not present in twitter api only 
# intersection rows this way we will have consistent row count between the 2 dataframes
print(df_api_clean.shape)
print(master_df_clean.shape)

master_df_clean = pd.merge(master_df_clean, df_api_clean,
                                   how = 'inner', on = 'tweet_id')

* ### Test

In [ ]:
master_df_clean.info()

* ### a.1.4 Define

After visually assessed the name column found a lot of records with "None" or "a" values and when checked
the text column for the names found it was even really missing or somehow really has no pattern to follow to extract the name 

* ### Code

In [ ]:
master_df_clean.query('name == "a" or name  == "None"').count()

In [ ]:
# here is a sample of the texts where the name = "a" or "None" 
# it shows that it's hard to get a specific pattern to extract name from it

pd.set_option('display.max_colwidth', -1)
master_df_clean.query('name == "a" or name  == "None" ')['text'].sample(10)


In [ ]:
# will change the "a" and "None" to nan

master_df_clean.loc[master_df_clean['name'] == 'None','name'] = np.nan
master_df_clean.loc[master_df_clean['name'] == 'a','name'] = np.nan


* ### Test

In [ ]:
# no records now with name = a or name  = None
master_df_clean.query('name == "a" or name  == "None"').count()

* ### a.1.5 Define

- Missing values in in dog classifications columns puppo, doggo, pupper ,floofer are None not nan


* ### Code

In [ ]:
# master_df_clean[master_df_clean.floofer.isnull()] 
# master_df_clean.loc[master_df_clean['doggo'] == 'doggo', 'dog_class'] = 'doggo'
# master_df_clean.loc[master_df_clean['floofer'] == 'floofer', 'dog_class'] = 'floofer'
# master_df_clean.loc[master_df_clean['pupper'] == 'pupper', 'dog_class'] = 'pupper'
# master_df_clean.loc[master_df_clean['puppo'] == 'puppo', 'dog_class'] = 'puppo'
# master_df_clean.info()
# master_df_clean[master_df_clean['doggo'] != 'None'].count()
master_df_clean.loc[master_df_clean['floofer'] == 'None', 'floofer']  = ''
master_df_clean.loc[master_df_clean['pupper'] == 'None', 'pupper']  = ''
master_df_clean.loc[master_df_clean['doggo'] == 'None', 'doggo']  = ''
master_df_clean.loc[master_df_clean['puppo'] == 'None', 'puppo']  = ''

* ### Test

In [ ]:
master_df_clean.info()
master_df_clean.sample(1)

### b. df_image_pred :

* ### b.1.1 Define

* Column names are not descriptive 


* ### Code

In [ ]:
master_df_clean.rename(columns = {'p1' : 'first_probability' ,'p2' : 'second_probability' ,'p3' : 'third_probability' },inplace = True)


* ### Test

In [ ]:
master_df_clean.head(3)

## 2. Inconsistency

* ### 2.1 Define

- Inconsistant Capitalization in prediction names

* ### Code


In [ ]:
master_df_clean['first_probability'] = master_df_clean['first_probability'].str.lower()
master_df_clean['second_probability'] = master_df_clean['second_probability'].str.lower()
master_df_clean['third_probability'] = master_df_clean['third_probability'].str.lower()


* ### Test


In [ ]:
master_df_clean[['first_probability','second_probability','third_probability']].sample(5) 


* ### 2.2 Define

Timestamp is of object type need to be converted to datetime 

* ### Code

In [ ]:

master_df_clean['timestamp'] = pd.to_datetime( master_df_clean['timestamp'])


* ### Test

In [ ]:
master_df_clean.info()

## 3. Accuracy

* ### 3.1 Define

rating numerators have some abnormal values up to 1776 which is not normal and need investigation

* ### Code

In [ ]:
master_df_clean.rating_numerator.describe()
master_df_clean.query('(rating_numerator > 20) or (rating_numerator < 5)').count()

In [ ]:
# here i queried all the rating numerators > 15 and numerator not equal 10 and denominator can be divided by 10
# this way we can manipulate the observation which might contain more than one dog on the same photo (observation)
# so will get a row for each dog with eaqually divided rating numerators.
master_df_clean.query('((rating_numerator > 15) ) and (rating_denominator > 10) and (rating_denominator % 10 == 0)')


In [ ]:
# Calculating the dogs counts in the tweets featuring many dogs
dog_count = master_df_clean.rating_denominator[master_df_clean.rating_denominator > 20 ]/10

# Check the dog_count with their indices:
print(dog_count)

# Performing the Calculations

master_df_clean.loc[(master_df_clean.rating_numerator >= 15) & (master_df_clean.rating_numerator >  10), ['rating_numerator', 'rating_denominator']] = [master_df_clean.rating_numerator[(master_df_clean.rating_numerator >= 15 & (master_df_clean.rating_numerator >  10))]/dog_count , 10]

* ### Test

In [ ]:
master_df_clean.query('(rating_numerator > 15)  and (rating_denominator != 10) ')

* ### 3.2 Define

- rating denominators have some abnormal data (while denominators should always be 10 ) for each dog observation.

* ### Code

In [ ]:
master_df_clean.query(' (rating_denominator != 10) ')

In [ ]:
master_df_clean.loc[(master_df_clean.rating_denominator != 10),['rating_denominator']] = 10

* ### Test

In [ ]:
master_df_clean.loc[(master_df_clean.rating_denominator != 10),['rating_denominator']]

## Tideiness Issues

* ### 1.1 Define

- variables are used as column names (doggo,floofer,pupper,puppo) 

* ### Code

In [ ]:
# add all the stages column into one new column so that a column will contain the variables values
master_df_clean['dog_stage'] = master_df_clean['doggo'] + master_df_clean['floofer'] + master_df_clean['pupper'] + master_df_clean['puppo']
# drop old columns 
master_df_clean.drop(columns  = ['puppo','pupper','doggo','floofer'],inplace = True)
# replace blank strings with nan
master_df_clean['dog_stage'] = master_df_clean.dog_stage.apply(lambda x: np.nan if  (x.isspace() or not x) else x)
# detect the values of dog_stages where 2 stages present in the same time (ex: puppodoggo)
master_df_clean.loc[master_df_clean['dog_stage'].str.len() > 7]
# if found 2 of them have values in the same row join them dash separated (ex : puppo-doggo) to be a value itself
for idx, row in master_df_clean[master_df_clean['dog_stage'].str.len() > 7].iterrows():
    dog_types = []
    if 'puppo' in row.text:
        dog_types.append('puppo')
    if 'pupper' in row.text:
        dog_types.append('pupper')
    if 'doggo' in row.text:
        dog_types.append('doggo')
    if 'floofer' in row.text:
        dog_types.append('floofer')
    if len(dog_types) > 0:
        master_df_clean.loc[idx, 'dog_stage'] = '-'.join(dog_types)
master_df_clean.dog_stage.value_counts(dropna = False)


* ### Test

In [ ]:
master_df_clean.info()
